<a href="https://colab.research.google.com/github/SylviaMwai/FitGuru-Classifier-Using-KNN/blob/main/FitGuru_Classifier_Using_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FitGuru Classifier USING KNN

In [1]:
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Define categories
time_of_day = ['Morning', 'Afternoon', 'Evening']
workout_type = ['Cardio', 'Strength']
intensity_levels = ['Low', 'Medium', 'High']

# Generate Workout_Type
workout_labels = np.random.choice(workout_type, size=50)

# Generate features
duration = [
    np.random.randint(60, 181) if wt == 'Strength' else np.random.randint(30, 61)
    for wt in workout_labels
]
heart_rate = np.random.randint(90, 160, size=50)
calories = np.random.randint(100, 700, size=50)
time = np.random.choice(time_of_day, size=50)
intensity = [
    np.random.choice(['Medium', 'High']) if wt == 'Strength' else np.random.choice(intensity_levels)
    for wt in workout_labels
]

# Create DataFrame
df = pd.DataFrame({
    'Duration': duration,
    'Avg_Heart_Rate': heart_rate,
    'Calories_Burned': calories,
    'Time_of_Day': time,
    'Workout_Type': workout_labels,
    'Workout_Intensity': intensity
})

# Save to CSV
df.to_csv('smart_workout_classifier_dataset.csv', index=False)
print("Dataset saved as 'smart_workout_classifier_dataset.csv'")


Dataset saved as 'smart_workout_classifier_dataset.csv'


In [2]:
Data = pd.read_csv('smart_workout_classifier_dataset.csv')
Data.head()

,Duration,Avg_Heart_Rate,Calories_Burned,Time_of_Day,Workout_Type,Workout_Intensity
0,44,133,674,Evening,Cardio,High
1,121,123,340,Morning,Strength,Medium
2,48,151,663,Afternoon,Cardio,Medium
3,41,103,195,Afternoon,Cardio,Medium
4,52,137,584,Afternoon,Cardio,High
